### 원본 데이터 불러오기

In [ ]:
import pickle
import pandas as pd

pothole_coor = pd.read_pickle("/content/drive/MyDrive/DATA_전처리/pothole_coordinates.pickle")
pothole_dongs = pd.read_pickle("/content/drive/MyDrive/DATA_전처리/pothole_dongs.pickle")
pothole_final = pd.read_pickle("/content/drive/MyDrive/DATA_전처리/pothole_final3.pickle")

### 교통량 병합

In [ ]:
import geopandas as gpd

In [ ]:
gdf_potholes = gpd.GeoDataFrame(
    pothole_final,
    geometry=gpd.points_from_xy(pothole_final['경도'], pothole_final['위도']),
    crs="EPSG:4326"  # 위/경도 WGS84 좌표계로 설정
)

In [ ]:
gdf_links = gpd.read_file('/content/drive/MyDrive/DATA_원본/[2024-02-23]NODELINKDATA/MOCT_LINK.shp', encoding='euc-kr')
# 링크 ID 컬럼을 숫자형으로 변환 (문자형일 경우 대비)
gdf_links['LINK_ID'] = gdf_links['LINK_ID'].astype(int)

# 1-3. 좌표계 통일: 포트홀 점들을 도로 링크의 좌표계로 변환 (링크 데이터는 평면 좌표계 사용)
if gdf_links.crs is not None:
    gdf_potholes = gdf_potholes.to_crs(gdf_links.crs)

In [ ]:
gdf_nearest = gpd.sjoin_nearest(
    gdf_potholes, gdf_links,
    how='left',
    distance_col='distance'  # 계산한 거리(미터) 확인
)

gdf_nearest[['LINK_ID', 'distance']]

In [ ]:
traffic_d = pd.read_excel('/content/drive/MyDrive/DATA_원본/TrafficVolume(LINK).xlsx', header=[0, 1])

In [ ]:
traffic_d.columns = [
    f"{upper}" if 'Unnamed' in str(lower) else f"{upper}_{lower}"
    for upper, lower in traffic_d.columns
]

In [ ]:
traffic_d.rename(columns={
    'ITS LINK ID': 'ITS_LINK_ID',
    '승용차-평일_전일': '승용차',
    '버스-평일_전일': '버스',
    '트럭-평일_전일': '트럭'
}, inplace=True)

# ITS_LINK_ID가 쉼표로 연결된 경우를 리스트로 변환
traffic_d['ITS_LINK_ID'] = traffic_d['ITS_LINK_ID'].astype(str).str.split(',')

# 각 링크 ID마다 하나의 행 생성
traffic_d = traffic_d.explode('ITS_LINK_ID')

# 링크 ID를 정수형으로 변환 (필요시 공백 제거 후 처리)
traffic_d['LINK_ID'] = traffic_d['ITS_LINK_ID'].str.strip().astype(int)

traffic_df = traffic_d[['LINK_ID', '도로명', '차선수', '승용차', '버스', '트럭']].copy()

In [ ]:
traffic_df = (
    traffic_df
    .dropna(subset=['LINK_ID'])                 # 결측 제거
    .drop_duplicates('LINK_ID')                # 중복 제거
    .reset_index(drop=True)                    # 인덱스 초기화
    .astype({'LINK_ID': 'int'})                # 정수형 확정
)

print(traffic_df.head())
print("✔️ LINK_ID 유니크 여부:", traffic_df['LINK_ID'].is_unique)

In [ ]:
# 1. 총교통량 = 승용차 + 버스 + 트럭
traffic_df['총교통량'] = traffic_df['승용차'] + traffic_df['버스'] + traffic_df['트럭']

# 2. 중대형차량 교통량 비율 = (버스 + 트럭) / 총교통량
traffic_df['중대형차량 교통량'] = (traffic_df['버스'] + traffic_df['트럭']) / traffic_df['총교통량']

traffic_df.head(10)

In [ ]:
# 포트홀 데이터와 교통량 데이터 병합 (링크 ID 기준)
pothole_traffic = gdf_nearest.merge(traffic_df, on='LINK_ID', how='outer')

pothole_traffic

In [ ]:
# 등록번호가 NaN인 행 추출
only_traffic = pothole_traffic['등록번호'].isna().sum()

print("포트홀 미발생지역 교통량 데이터 수:", only_traffic)

In [ ]:
# 포트홀 데이터 중 총교통량이 누락된 경우가 있는지 확인
missing_traffic_info = pothole_traffic['총교통량'].isna().sum()

print("교통량이 없는 포트홀 위치:", missing_traffic_info)

### 포트홀 미발생지역 좌표변환

In [ ]:
# 포트홀 발생 정보를 나타내는 컬럼(예: '등록번호')가 NaN인 경우, 포트홀이 없는 데이터
no_pothole_df = pothole_traffic[pothole_traffic['등록번호'].isna()].copy()

# 포트홀이 없는 데이터에서 고유한 LINK_ID 추출
traffic_link_ids = no_pothole_df['LINK_ID'].unique()

# 전체 도로 링크 셰이프파일(gdf_link)에서, 포트홀이 없는 데이터에 해당하는 LINK_ID만 필터링
gdf_link_subset = gdf_links[gdf_links['LINK_ID'].isin(traffic_link_ids)].copy()

if gdf_link_subset.crs != "EPSG:4326":
    gdf_link_subset = gdf_link_subset.to_crs("EPSG:4326")

# 각 도로 링크의 대표 좌표(centroid)를 계산하고, 경도(lon)와 위도(lat) 추출
gdf_link_subset['centroid'] = gdf_link_subset.geometry.centroid
gdf_link_subset['lon'] = gdf_link_subset.centroid.x
gdf_link_subset['lat'] = gdf_link_subset.centroid.y

pothole_traffic = pothole_traffic.merge(
    gdf_link_subset[['LINK_ID', 'lon', 'lat']],
    on='LINK_ID',
    how='left',
    suffixes=('', '_추가')  # 기존 lat/lon과 충돌하지 않게
)

In [ ]:
# lan lon을 경위도로 대체
pothole_traffic['경도'] = pothole_traffic['경도'].combine_first(pothole_traffic['lon'])
pothole_traffic['위도'] = pothole_traffic['위도'].combine_first(pothole_traffic['lat'])

pothole_traffic.drop(columns=['lon', 'lat'], inplace=True)
# pothole_traffic.to_excel("포트홀 교통량.xlsx", index=False)

### 교통량 보간

트럭 통행제한 도로의 경우 트럭, 버스 교통량 데이터가 결측 -> 제거

In [ ]:
pothole_traffic['REST_VEH'] = pd.to_numeric(pothole_traffic['REST_VEH'], errors='coerce')

In [ ]:
pothole_traffic = pothole_traffic.query('not ((REST_VEH == 4) and 트럭.isnull())')

다차선 도로 보간

In [ ]:
pothole_traffic.query(
    'LANES >= 2 and 총교통량.isnull()')

In [ ]:
import math
from tqdm import tqdm

# 1. Haversine 거리 계산 함수
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, (lon1, lat1, lon2, lat2))
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return 2 * 6371000 * math.asin(math.sqrt(a))  # m 단위

In [ ]:
# 2. 후보군(교통량이 있는 다차선 도로) 추출
cand_mask = (
    (pothole_traffic['LANES'] >= 2) &
    pothole_traffic[['승용차','버스','트럭']].notna().all(axis=1)
)
candidates  = pothole_traffic[cand_mask].copy()
cand_coords = candidates[['위도','경도']].to_numpy()
cand_vals   = candidates[['승용차','버스','트럭']].to_numpy()

# 3. 보간 대상(교통량이 하나라도 NaN인 다차선 도로)
mask_missing = (
    (pothole_traffic['LANES'] >= 2) &
    (
        pothole_traffic['승용차'].isnull() |
        pothole_traffic['버스'].isnull() |
        pothole_traffic['트럭'].isnull()
    )
)

In [ ]:
# 4. 거리 기반 보간 수행
for idx in tqdm(pothole_traffic[mask_missing].index, desc="보간 중"):
    row = pothole_traffic.loc[idx]
    lat0, lon0 = row['위도'], row['경도']
    if pd.isna(lat0) or pd.isna(lon0):
        continue  # 위치 정보가 없으면 건너뜀

    # 거리 계산
    dists = np.array([
        haversine(lon0, lat0, lon1, lat1)
        for lat1, lon1 in cand_coords
    ])
    dists[dists == 0] = 1e-3  # 자기 자신일 경우 방지

    # 최근접 5개 인덱스 및 가중치 계산
    nn   = np.argsort(dists)[:5]
    vals = cand_vals[nn]
    ws   = 1 / dists[nn]
    ws  /= ws.sum()

    # 보간값 계산
    imputed = np.dot(ws, vals)

    # 결측된 값만 채움
    for i, col in enumerate(['승용차', '버스', '트럭']):
        if pd.isna(pothole_traffic.at[idx, col]):
            pothole_traffic.at[idx, col] = imputed[i]

# 5. 총교통량 및 중대형차량 교통량 계산
pothole_traffic['총교통량'] = (
    pothole_traffic['승용차'] + pothole_traffic['버스'] + pothole_traffic['트럭']
)

pothole_traffic['중대형차량 교통량'] = (
    (pothole_traffic['버스'] + pothole_traffic['트럭']) / pothole_traffic['총교통량']
)

1차선 도로 보간

In [ ]:
# 1. 차선수별 평균 교통량 계산
lane_groups = pothole_traffic.groupby('차선수')[['승용차','버스','트럭']].mean()
lane_groups

In [ ]:
# 2. 1차선 대비 다차선 비율 계산 (1차선 평균 / n차선 평균)
#    예: ratio_car[4] = avg(승용차@1차선) / avg(승용차@4차선)
ratio_car  = lane_groups.loc[1,'승용차'] / lane_groups['승용차']
ratio_bus  = lane_groups.loc[1,'버스']   / lane_groups['버스']
ratio_truck= lane_groups.loc[1,'트럭']   / lane_groups['트럭']

ratio_car # 인접한 도로가 6차선 도로라면 그 도로 교통량의 0.58을 곱한 값을 1차선 도로의 승용차 교통량으로 보간

In [ ]:
# 2. 1차선 대비 다차선 비율 계산 (1차선 평균 / n차선 평균)
#    예: ratio_car[4] = avg(승용차@1차선) / avg(승용차@4차선)
ratio_car  = lane_groups.loc[1,'승용차'] / lane_groups['승용차']
ratio_bus  = lane_groups.loc[1,'버스']   / lane_groups['버스']
ratio_truck= lane_groups.loc[1,'트럭']   / lane_groups['트럭']

ratio_car # 인접한 도로가 6차선 도로라면 그 도로 교통량의 0.58을 곱한 값을 1차선 도로의 승용차 교통량으로 보간

In [ ]:
# 4. 보간 대상: “LANES==1” & 하나라도 NaN

mask_missing = (pothole_traffic['LANES']==1) & (pothole_traffic[['승용차','버스','트럭']].isna().any(axis=1))

In [ ]:
# 5. 보간 루프
for idx in tqdm(pothole_traffic[mask_missing].index, desc="1차선 비율보간"):
    row = pothole_traffic.loc[idx]
    lat0, lon0 = row['위도'], row['경도']
    if pd.isna(lat0) or pd.isna(lon0):
        continue

    # 가장 인접한 도로 선택
    dists = np.array([haversine(lon0, lat0, lon1, lat1)
                      for lat1, lon1 in cand_coords])
    nn = np.argmin(dists)

    n_lane = cand_lane[nn]       # ex: 4차선
    vals   = cand_vals[nn]       # ex: [400, 30, 40]

    # 1차선 교통량 = 다차선 교통량 × (1차선평균/다차선평균)
    imputed_car   = vals[0] * ratio_car.loc[n_lane]
    imputed_bus   = vals[1] * ratio_bus.loc[n_lane]
    imputed_truck = vals[2] * ratio_truck.loc[n_lane]

    # NaN인 컬럼에만 채우기
    if pd.isna(row['승용차']):    pothole_traffic.at[idx,'승용차'] = imputed_car
    if pd.isna(row['버스']):      pothole_traffic.at[idx,'버스']   = imputed_bus
    if pd.isna(row['트럭']):      pothole_traffic.at[idx,'트럭']   = imputed_truck

# 6. 파생변수 재계산
pothole_traffic['총교통량'] = pothole_traffic[['승용차','버스','트럭']].sum(axis=1)
pothole_traffic['중대형차량 교통량'] = (
    (pothole_traffic['버스']+pothole_traffic['트럭']) /
    pothole_traffic['총교통량']
)

In [ ]:
# 경위도가 있는 값 중 교통량 보간여부 확인
pothole_traffic[(pothole_traffic['위도'].notna()) & (pothole_traffic['총교통량'].isna())]

In [ ]:
pothole_traffic[pothole_traffic[['승용차', '버스', '트럭']].isna().any(axis=1)]

### 건축물연령

In [ ]:
age = pd.read_csv('/content/drive/MyDrive/DATA_원본/건축물연령정보_행정동_평균.csv')

In [ ]:
# 1) 결측치 제외하고 '행정동'의 고유값(Unique values) 보기
unique_dongs = pothole['행정동'].dropna().unique()
print(unique_dongs)

# 2) 정렬된(unique) 리스트로 보기
unique_dongs_sorted = sorted(unique_dongs)
print(unique_dongs_sorted)

In [ ]:
unique_dongs = age['동명'].dropna().unique()
print(unique_dongs)

In [ ]:
import pandas as pd
import re

# 편의를 위해 age의 동명 목록을 집합으로 생성 (법정동 표준명 리스트)
legal_names = set(age['동명'])

# 행정동 → 법정동 정규화 함수 정의
def normalize_dong(name: str) -> str:
    if pd.isna(name):  # 결측값 처리
        return name
    name = str(name).strip()
    # 1. 특수 구분자 통일 ('.', '·', 'ㆍ' 등을 '.''로 변환)
    name = name.replace('·', '.').replace('ㆍ', '.')

    # 2. 사전 매핑 정의 (주요 복합 행정동 → 대표 법정동)
    mapping = {
        '종로1.2.3.4가동': '종로1가',
        '종로1·2·3·4가동': '종로1가',
        '종로5.6가동': '종로5가',
        '종로5·6가동': '종로5가',
        '청운효자동': '청운동',
        '왕십리도선동': '하왕십리동',
        '성수1가동': '성수동1가',
        '성수2가동': '성수동2가',
        '여의동': '여의도동',
        '원효로동': '원효로1가',
        '원효로제2동' : '원효로4가',
        '용산동4가' : '용산동3가',
        '성수2가제1동' : '성수동2가',
        '성수2가제2동' : '성수동1가',
        '성수1가제1동' : '성수동1가',
        '성수1가제2동' : '성수동2가',
        '성수1가제3동' : '성수동1가',
        '성수2가제3동' : '성수동2가',
        '낙성대동' : '봉천동',
        '다산동' : '신당동'
    }
    if name in mapping:
        return mapping[name]

    # 3. 이미 정규화된 법정동 명칭일 경우 바로 반환
    if name in legal_names:
        return name

    # 4. '제#동' 패턴 처리: '제' 제거 (예: '제1동' -> '1동')
    name = re.sub(r'제(?=\d+동$)', '', name)

    # 5. '...가동' 패턴 처리
    if name.endswith('가동'):
        # 정규식으로 접두어와 첫 번째 숫자 추출
        m = re.match(r'^(.+?)(\d+).*가$', name)
        if m:
            prefix, first_num = m.group(1), m.group(2)
            # 접두어에 해당하는 법정동이 존재하면 접두어+'동' 사용
            candidate = prefix + '동' + first_num + '가'
            if candidate in legal_names:
                return candidate
            else:
                return prefix + first_num + '가'

    # 6. '~동'으로 끝나고 숫자가 포함된 패턴 처리 (예: '숭인2동' -> '숭인동')
    if re.match(r'.*\d+동$', name):
        name = re.sub(r'\d+동$', '동', name)  # 마지막 숫자 제거

        # 제거 후 바로 표준 법정동이면 반환
        if name in legal_names:
            return name

    # 7. 숫자 없음 + 법정동 1가 패턴 처리 (예: '명동' -> '명동1가')
    if name.endswith('동') and (name + '1가') in legal_names:
        return name + '1가'

    # 8. 기타: 처리 후 legal_names에 있지 않으면 그대로 반환하거나 NaN 처리
    return name

# pothole 데이터프레임에 새로운 열로 정규화된 동명 추가
pothole['행정동_정규화'] = pothole['행정동'].apply(normalize_dong)

# 9. age 데이터의 평균_건물연령 값을 pothole 데이터에 병합
pothole = pothole.merge(age[['동명', '평균_건물연령']],
                        left_on='행정동_정규화', right_on='동명', how='left')

# 필요시 '동명' 열 제거 및 정리
pothole = pothole.drop(columns=['동명'])  # 병합으로 생긴 불필요한 열 제거

In [ ]:
# 포트홀 존재여부==1 이면서 평균_건물연령 결측인 행
mask = pothole['평균_건물연령'].isna()
missing_age = pothole[mask]

# 결과 확인
print(missing_age.shape)
missing_age

In [ ]:
# '평균건물_연령'이 NaN인 행 제거
pothole = pothole.dropna(subset=['평균_건물연령'])

In [ ]:
dup_mask = pothole['index'].duplicated(keep=False)

# 2) 중복된 모든 행을 'index' 기준으로 정렬하여 확인
duplicated_rows = pothole.loc[dup_mask].sort_values('index')

# 3) 결과 출력
print(duplicated_rows)

In [ ]:
# 'index' 컬럼 기준 중복 제거 (첫 번째만 남김), 원본 pothole 갱신
pothole.drop_duplicates(subset='index', keep='first', inplace=True)

# (선택) 인덱스 정리도 함께 하고 싶다면
pothole.reset_index(drop=True, inplace=True)

In [ ]:
pothole.drop(columns=['index'], inplace=True)

#### 기온, 습도, 강수량, 인구 수 전처리

In [ ]:
### 기온, 강수
temp_all = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_기온강수.csv", encoding = 'cp949')
temp_jong = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_기온강수.csv", encoding = 'cp949')

### 습도
humid_all_1 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_습도_2101_2110.csv", encoding = 'cp949')
humid_all_2 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_습도_2111_2210.csv", encoding = 'cp949')
humid_all_3 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_습도_2211_2310.csv", encoding = 'cp949')
humid_all_4 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_습도_2311_2410.csv", encoding = 'cp949')
humid_all_5 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/자치구별_습도_2411_2504.csv", encoding = 'cp949')

humid_jong_1 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_습도_2101_2110.csv", encoding = 'cp949')
humid_jong_2 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_습도_2111_2210.csv", encoding = 'cp949')
humid_jong_3 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_습도_2211_2310.csv", encoding = 'cp949')
humid_jong_4 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_습도_2311_2410.csv", encoding = 'cp949')
humid_jong_5 = pd.read_csv("/content/drive/MyDrive/DATA_원본/자치구별 데이터/종로구_습도_2411_2504.csv", encoding = 'cp949')

### 인구 수
people = pd.read_excel("/content/drive/MyDrive/DATA_원본/자치구별 인구.xlsx")

- 기온, 강수

In [ ]:
temp_all['지점명'].unique()
temp_all['지점명'] = temp_all['지점명'].replace({
    "중구" : "중",
    "강북*" : "강북"
})
temp_all['지점명'].unique()

In [ ]:
### 전체 + 종로구 합치기
temp_jong['지점명'] = temp_jong['지점명'].replace({"서울" : "종로"})
temp = pd.concat([temp_all,temp_jong], ignore_index = True)

### 기상청, 현충원 합쳐서 동작구로 넣기
# 기상청 - 일강수량 결측치 하나
# 현충원 - 기온 결측치 5개, 일강수량 하나
filtered = temp[temp['지점명'].isin(["기상청", "현충원"])].drop(labels = "지점명", axis = 1)
dongjak = filtered.groupby('일시').mean().reset_index()
dongjak['지점명'] = '동작'
dongjak = dongjak[['지점명', '일시', '평균기온(°C)',	'최저기온(°C)',	'최고기온(°C)', '일강수량(mm)']]
temp = pd.concat([temp, dongjak], ignore_index = True)

### 자치구 필터링
gus = ["강남", "강동", "강북", "강서", "관악", "광진", "구로", "금천", "노원",
       "도봉", "동대문", "동작", "마포", "서대문", "서초", "성동", "성북",
       "송파", "양천", "영등포", "용산", "은평", "종로", "중", "중랑"]
temp = temp[temp['지점명'].isin(gus)]
temp['지점명'].nunique() # 25개 다 있다!!

In [ ]:
### 데이터 보간
all_dates = pd.date_range(start = '2021-01-01', end = '2025-04-30', freq = 'D')

def fill_dates(gu):
    gu_data = gu.set_index('일시')
    gu_data.index = pd.to_datetime(gu_data.index)
    # 비어있는 날짜 채우기
    gu_data = gu_data.reindex(all_dates) # 새로 index 생성
    # 보간하기!
    gu_data['지점명'] = gu_data['지점명'].fillna(gu['지점명'].iloc[0])
    # 평균기온
    gu_data['평균기온(°C)'] = gu_data['평균기온(°C)'].interpolate(method = 'nearest', limit_direction = 'both')
    gu_data['평균기온(°C)'] = gu_data['평균기온(°C)'].fillna(gu['평균기온(°C)'].mean())
    # 최저기온
    gu_data['최저기온(°C)'] = gu_data['최저기온(°C)'].interpolate(method = 'nearest', limit_direction = 'both')
    gu_data['최저기온(°C)'] = gu_data['최저기온(°C)'].fillna(gu['최저기온(°C)'].mean())
    # 최고기온
    gu_data['최고기온(°C)'] = gu_data['최고기온(°C)'].interpolate(method = 'nearest', limit_direction = 'both')
    gu_data['최고기온(°C)'] = gu_data['최고기온(°C)'].fillna(gu['최고기온(°C)'].mean())
    # 일강수량
    gu_data['일강수량(mm)'] = gu_data['일강수량(mm)'].interpolate(method = 'nearest', limit_direction = 'both')
    gu_data['일강수량(mm)'] = gu_data['일강수량(mm)'].fillna(0)
    return gu_data.reset_index()

filled_temp = temp.groupby('지점명').apply(fill_dates).reset_index(drop = True)
filled_temp.columns = ['일시', '지점', '지점명', '평균기온(°C)','최저기온(°C)',	'최고기온(°C)', '일강수량(mm)']
temp = filled_temp
temp.head()

In [ ]:
### 마지막 전처리
# '구' 추가하기
temp['지점명'] = temp['지점명'].astype(str) + '구'
# '지점' 변수 삭제
temp.drop(labels = '지점', axis = 1, inplace = True)
# '일시' 변수 시간 변수로 변경
temp['일시'] = pd.to_datetime(temp['일시'])
# 강수 drop
tempp = temp.drop(labels = '일강수량(mm)', axis = 1)
tempp.head()

- 습도

In [ ]:
humid_all = pd.concat([humid_all_1, humid_all_2, humid_all_3, humid_all_4, humid_all_5], ignore_index = True)
humid_jong = pd.concat([humid_jong_1, humid_jong_2, humid_jong_3, humid_jong_4, humid_jong_5], ignore_index = True)
humid_all.head()

In [ ]:
### 앞의 과정 반복...
# 이름 바꿔주고
humid_all['지점명'] = humid_all['지점명'].replace({
    "중구" : "중",
    "강북*" : "강북"
})
# 종로구 합쳐주고
humid_jong['지점명'] = humid_jong['지점명'].replace({"서울" : "종로"})
humid = pd.concat([humid_all,humid_jong], ignore_index = True)
# 기상청, 현충원 합쳐서 동작구로
filtered = humid[humid['지점명'].isin(["기상청", "현충원"])].drop(labels = "지점명", axis = 1)
dongjak = filtered.groupby('일시').mean().reset_index()
dongjak['지점명'] = '동작'
dongjak = dongjak[['지점명', '일시', '습도(%)']]
humid = pd.concat([humid, dongjak], ignore_index = True)

# 자치구 필터링
gus = ["강남", "강동", "강북", "강서", "관악", "광진", "구로", "금천", "노원",
       "도봉", "동대문", "동작", "마포", "서대문", "서초", "성동", "성북",
       "송파", "양천", "영등포", "용산", "은평", "종로", "중", "중랑"]
humid = humid[humid['지점명'].isin(gus)]
print(humid['지점명'].nunique()) # 25개 다 있다!!

# '구' 추가, '지점' 삭제, '일시' 시간 변수로
humid['지점명'] = humid['지점명'].astype(str) + '구'
humid.drop(labels = '지점', axis = 1, inplace = True)
humid['일시'] = pd.to_datetime(humid['일시'])
humid.head(3)

In [ ]:
humid['날짜'] = pd.to_datetime(humid['일시']).dt.date
humid_group = humid.groupby(['지점명','날짜'])['습도(%)'].mean().reset_index()
humid = humid_group
humid.head(2)

In [ ]:
all_dates = pd.date_range(start = '2021-01-01', end = '2025-04-30', freq = 'D')

def fill_dates(gu):
    mean_humidity = gu['습도(%)'].mean()
    gu_data = gu.set_index('날짜')
    # 비어있는 날짜 채우기
    gu_data = gu_data.reindex(all_dates) # 새로 index 생성
    # 보간하기!
    gu_data['습도(%)'] = gu_data['습도(%)'].interpolate(method = 'nearest', limit_direction = 'both')
    gu_data['습도(%)'] = gu_data['습도(%)'].fillna(mean_humidity)
    gu_data['지점명'] = gu_data['지점명'].fillna(gu['지점명'].iloc[0])
    return gu_data.reset_index()

filled_humid = humid.groupby('지점명').apply(fill_dates).reset_index(drop = True)
filled_humid.columns = ['날짜', '자치구', '습도(%)']
humid = filled_humid
humid.head()

In [ ]:
temp.head()
rain = temp[['지점명','일시','일강수량(mm)']]
rain.head()

- 인구 수

In [ ]:
people = people.drop(index = 0)
people['평균 인구 수'] = people.drop(columns = ['행정구역(시군구)별']).mean(axis = 1)
people = people[['행정구역(시군구)별', '평균 인구 수']]
people.columns = ['자치구', '인구 수']
people.head()

In [ ]:
people['자치구'] = people['자치구'].str.replace("\u3000","",regex = False)

### 기온, 습도, 강수량, 인구 수 합산

In [ ]:
### 기록된 날짜 기준 한 달 전으로 '발생일' 생성
small_pothole['발생일'] = small_pothole['등록번호'].astype(str).str[:8]
small_pothole['발생일'] = pd.to_datetime(small_pothole['발생일'])
small_pothole['발생일'] = small_pothole['발생일'].apply(lambda x:x - pd.Timedelta(days = 30))
small_pothole.head(2)

- 인구 수

In [ ]:
### 인구 수
people_merged = pd.merge(small_pothole, people, on = '자치구', how = 'left')
people_merged.head(2)

- 강수량

In [ ]:
### 자치구 별, 발생일 기준 1년동안의 누적 강수량 합계
rain_sum = rain.copy()
# 각 자치구별로 누적합 - 약 1분 소요
def cumulate_rain(gu):
    for i, row in gu.iterrows():
        end = pd.to_datetime(row['일시'])
        start = end - pd.Timedelta(days = 365)
        subset = gu[(gu['일시'] >= start) & (gu['일시'] <= end)]
        gu.loc[i, '누적 강수량'] = subset['일강수량(mm)'].sum()
    return gu
rain_sum = rain_sum.groupby('지점명').apply(cumulate_rain).reset_index(drop = True)
rain_sum = rain_sum.drop(labels = '일강수량(mm)', axis = 1)
rain_sum.columns = ['자치구', '발생일','누적 강수량']
rain_sum.head()

In [ ]:
rain_trying = people_merged.copy()
# rain_trying['1년 전'] = rain_trying['발생일'].apply(lambda x:x - pd.Timedelta(days = 365))

rain_trying = pd.merge(rain_trying, rain_sum, on = ['자치구', '발생일'], how = 'left')
rain_trying.head()

- 습도

In [ ]:
### 자치구 별, 발생일 기준 1년동안의 누적 습도의 평균
humid_mean = humid.copy()
# 각 자치구별로 누적합 - 약 1분 소요
def mean_humid(gu):
    for i, row in gu.iterrows():
        end = pd.to_datetime(row['날짜'])
        start = end - pd.Timedelta(days = 365)
        subset = gu[(gu['날짜'] >= start) & (gu['날짜'] <= end)]
        gu.loc[i, '누적 습도'] = subset['습도(%)'].mean()
    return gu
humid_mean = humid_mean.groupby('자치구').apply(mean_humid).reset_index(drop = True)
humid_mean = humid_mean.drop(labels = '습도(%)', axis = 1)
humid_mean.columns = ['발생일', '자치구','평균 습도']
humid_mean.head()

In [ ]:
# 정확히 계산되었는지 확인
end = pd.to_datetime(rain_trying['발생일'][0])
start = end - pd.Timedelta(days = 365)
subset = humid[(humid['날짜'] >= start) & (humid['날짜'] <= end) & (humid['자치구'] == '강서구')]
subset['습도(%)'].mean()

- 기온

In [ ]:
temp['일교차'] = temp['최고기온(°C)'] - temp['최저기온(°C)']
temp.head(2)

In [ ]:
### 방법 1 - 자치구 별, 발생일 기준 1년동안의 최고 기온 - 최저 기온
temp_total_diff = temp.copy()
# 각 자치구별로 최고 - 최저
def total_diff_temp(gu):
    for i, row in gu.iterrows():
        end = pd.to_datetime(row['일시'])
        start = end - pd.Timedelta(days = 365)
        subset = gu[(gu['일시'] >= start) & (gu['일시'] <= end)]
        gu.loc[i, '1년 기온차'] = subset['평균기온(°C)'].max() - subset['평균기온(°C)'].min()
    return gu
temp_total_diff = temp_total_diff.groupby('지점명').apply(total_diff_temp).reset_index(drop = True)
temp_total_diff = temp_total_diff.drop(labels = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일교차'], axis = 1)
temp_total_diff.columns = ['발생일', '자치구','1년 기온차']
temp_total_diff.head()

In [ ]:
temp_trying = pd.merge(temp_trying, temp_total_diff, on = ['자치구', '발생일'], how = 'left')
temp_trying.head()

In [ ]:
### 방법 2 - 자치구 별, 발생일 기준 한 달동안의 일교차 평균
temp_monthly_diff = temp.copy()
# 각 자치구별로 일교차의 평균
def monthly_diff_temp(gu):
    for i, row in gu.iterrows():
        end = pd.to_datetime(row['일시'])
        start = end - pd.Timedelta(days = 30)
        subset = gu[(gu['일시'] >= start) & (gu['일시'] <= end)]
        gu.loc[i, '평균 일교차'] = subset['일교차'].mean()
    return gu
temp_monthly_diff = temp_monthly_diff.groupby('지점명').apply(monthly_diff_temp).reset_index(drop = True)
temp_monthly_diff = temp_monthly_diff.drop(labels = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일교차'], axis = 1)
temp_monthly_diff.columns = ['발생일', '자치구','평균 일교차']
temp_monthly_diff.head()

In [ ]:
temp_trying = pd.merge(temp_trying, temp_monthly_diff, on = ['자치구', '발생일'], how = 'left')
temp_trying.head()

### 기온, 습도, 강수량, 인구 수 병합

In [ ]:
### 발생일 비어있는 경우를 위한...
rain_forna = rain_sum.groupby('자치구')['누적 강수량'].mean()
humid_forna = humid_mean.groupby('자치구')['평균 습도'].mean()
temp_total_diff_forna = temp_total_diff.groupby('자치구')['1년 기온차'].mean()
temp_monthly_diff_forna = temp_monthly_diff.groupby('자치구')['평균 일교차'].mean()

In [ ]:
# NaN은 그대로 두고, 나머지만 문자열로 변환
pothole['등록번호'] = pothole['등록번호'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)

In [ ]:
### 기록된 날짜 기준 한 달 전으로 '발생일' 생성
wrong = pothole['등록번호'].isna() | (~pothole['등록번호'].astype(str).str.startswith("2"))
pothole['발생일'] = pd.to_datetime(pothole['등록번호'].astype(str).where(~wrong).str[:8])
pothole['발생일'] = pothole['발생일'].apply(lambda x:x - pd.Timedelta(days = 30))
pothole.head(2)

In [ ]:
### 인구 수
people_merged = pd.merge(pothole, people, on = '자치구', how = 'left')
people_merged.head(2)

In [ ]:
with_date = people_merged[people_merged['발생일'].notna()].copy()
without_date = people_merged[people_merged['발생일'].isna()].copy()

In [ ]:
with_date_1 = pd.merge(with_date, rain_sum, on = ['자치구', '발생일'], how = 'left')
with_date_2 = pd.merge(with_date_1, humid_mean, on = ['자치구', '발생일'], how = 'left')
with_date_3 = pd.merge(with_date_2, temp_total_diff, on = ['자치구', '발생일'], how = 'left')
with_date_4 = pd.merge(with_date_3, temp_monthly_diff, on = ['자치구', '발생일'], how = 'left')
with_date_4.head()

In [ ]:
without_date_1 = pd.merge(without_date, rain_forna, on = ['자치구'], how = 'left')
without_date_2 = pd.merge(without_date_1, humid_forna, on = ['자치구'], how = 'left')
without_date_3 = pd.merge(without_date_2, temp_total_diff_forna, on = ['자치구'], how = 'left')
without_date_4 = pd.merge(without_date_3, temp_monthly_diff_forna, on = ['자치구'], how = 'left')
without_date_4.head()

In [ ]:
pothole_merged = pd.concat([with_date_4, without_date_4])
pothole_merged.head()

In [ ]:
pothole_merged.to_pickle('/content/drive/MyDrive/DATA_완성/준희 완료!.pickle')

### 배수등급 병합

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
# 1. 포트홀 데이터 로딩
pothole_df = gpd.GeoDataFrame(
    pothole_df,
    geometry=gpd.points_from_xy(pothole_df['경도'], pothole_df['위도']),
    crs="EPSG:4326"  # 위경도 좌표계
)

# 2. 포트홀 좌표계 → EPSG:5174 로 변환 (shp에 맞추기)
pothole_df = pothole_df.to_crs("EPSG:5174")

In [ ]:
# 3. 배수등급 shp 파일 로딩
soil_df = gpd.read_file("ASIT_SOILDRA_AREA.shp")  # 너가 가진 shp 파일명으로 대체
soil_df

In [ ]:
# 4. 공간 조인
pothole_df = gpd.sjoin(pothole_df, soil_df[['SOILDRA', 'geometry']], how='left', predicate='within')

# 결과 확인
pothole_df

### 토양경사도 병합

In [ ]:
# 3. 토양경사도 shp 파일 로딩
slope_df = gpd.read_file("ASIT_SOILSLOPE_AREA.shp")  # 너가 가진 shp 파일명으로 대체
slope_df

In [ ]:
pothole_df.drop(['index_right'], axis=1, inplace=True)

In [ ]:
# 4. 공간 조인
pothole_df = gpd.sjoin(pothole_df, slope_df[['SOILSLOPE', 'geometry']], how='left', predicate='within')

# 결과 확인
pothole_df

In [ ]:
pothole_df.drop(['index_right'], axis=1, inplace=True)
pothole_df.drop(['geometry'], axis=1, inplace=True)

In [ ]:
pothole_df.columns = ['자치구', '행정동', '경도', '위도', '차선수', '승용차', '버스', '트럭', '총교통량', '중대형차량 교통량',
       '포트홀 존재여부', '평균_건물연령', '등록번호', '발생일', '인구 수', '누적 강수량', '평균 습도',
       '1년 기온차', '평균 일교차', '배수등급', '경사도']